# Importing the House Price data from the Melbourne City Database (API)
(VB)

In [1]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
# base URL for API
base_url = "https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/house-prices-by-small-area-sale-year/records?limit=50&refine=sale_year%3A"

In [3]:
# get all years
min_year = 2000 - 1
max_year = 2016
temp = []

for i in range(max_year, min_year, -1):
    response = requests.get(base_url + str(i)).json()

    year_results = response["results"]

    for j in range(len(year_results)):
        temp.append(year_results[j])
    
house_price_df = pd.DataFrame(temp)
house_price_df.head()

,sale_year,small_area,type,median_price,transaction_count
0,2016,East Melbourne,House/Townhouse,1940000.0,35
1,2016,East Melbourne,Residential Apartment,650000.0,129
2,2016,Melbourne (CBD),House/Townhouse,NaN,2
3,2016,Melbourne (CBD),Residential Apartment,435000.0,1264
4,2016,North Melbourne,Residential Apartment,515000.0,231


In [4]:
# Get data from DF
house_price_df.count()

sale_year            347
small_area           347
type                 347
median_price         319
transaction_count    347
dtype: int64

In [5]:
# Drop NA data
house_price_df = house_price_df.dropna(how='any')
house_price_df.count()

sale_year            319
small_area           319
type                 319
median_price         319
transaction_count    319
dtype: int64

In [6]:
# Display clean DF
house_price_df.head()

,sale_year,small_area,type,median_price,transaction_count
0,2016,East Melbourne,House/Townhouse,1940000.0,35
1,2016,East Melbourne,Residential Apartment,650000.0,129
3,2016,Melbourne (CBD),Residential Apartment,435000.0,1264
4,2016,North Melbourne,Residential Apartment,515000.0,231
5,2016,Parkville,House/Townhouse,1761250.0,32


In [7]:
# Get unique small areas
print(house_price_df["small_area"].unique())

['East Melbourne' 'Melbourne (CBD)' 'North Melbourne' 'Parkville'
 'South Yarra' 'Southbank' 'Carlton' 'West Melbourne (Residential)'
 'Docklands' 'Kensington' 'Melbourne (Remainder)' 'NA']


In [24]:
# Modifying house_price_df for 2 year ranges, to compare with Income Data
starting_years = list(range(2003,2015 + 1,2))

# empty list to store new dictionaries
houseTH_median_price_two_year_dict = {}
houseTH_sum_transactions_two_year_dict = {}
ResApt_median_price_two_year_dict = {}
ResApt_sum_transactions_two_year_dict = {}

# loop through year ranges
for year in starting_years:
    st_year = house_price_df[house_price_df["sale_year"] == str(year)]
    en_year = house_price_df[house_price_df["sale_year"] == str(year + 1)]

    # get intersection
    two_year = pd.merge(st_year, en_year, how ='inner', on =['small_area', 'type']) 
    average_2y = (two_year["median_price_x"] + two_year["median_price_y"])/2

    # get rid of "Docklands House/Townhouse" and "Southbank House/Townhouse"

    # for House/Townhouses
    # add together transaction counts, and average median prices
    
    # for Residential Apartment
    # add together transaction counts, and average median prices

    
    # make new dictionary - merge all dictionaries later
    temp = {}
    #temp.[f"%s-%s" % (year, year + 1)] = average_2y["small_area"].tolist()
    
    print(two_year[["sale_year_x","small_area","type"]])
    print("\n\n")


   sale_year_x                    small_area                   type
0         2003                       Carlton  Residential Apartment
1         2003                     Docklands  Residential Apartment
2         2003                East Melbourne  Residential Apartment
3         2003                     Southbank  Residential Apartment
4         2003  West Melbourne (Residential)  Residential Apartment
5         2003                    Kensington  Residential Apartment
6         2003               Melbourne (CBD)  Residential Apartment
7         2003         Melbourne (Remainder)  Residential Apartment
8         2003                     Parkville        House/Townhouse
9         2003  West Melbourne (Residential)        House/Townhouse
10        2003                       Carlton        House/Townhouse
11        2003                East Melbourne        House/Townhouse
12        2003                    Kensington        House/Townhouse
13        2003               North Melbourne    

# Importing the Household Income data from the ABS
(LM)

In [8]:
# Specify the path to your Excel file
household_income_data = "Resources/5204055011do001.xlsx"  

In [9]:
# Define the row indices to skip
rows_to_skip = list(range(0, 6)) + [7] + list(range(9, 26)) + list(range(27, 40)) + list(range(42, 127))  

In [10]:
# Read ABS income data file and store into Pandas DataFrames
dfs = pd.read_excel(household_income_data, sheet_name=None, skiprows=rows_to_skip)  

In [11]:
# Create an empty list to store DataFrames with tab names as a column
dfs_with_tab_name = [] 

In [12]:
# Flag to skip the first and last tab
skip_first_tab = True
skip_last_tab = False

In [13]:
# Iterate over each sheet in the dictionary
for sheet_name, df in dfs.items():
    if skip_first_tab:
        skip_first_tab = False
        continue  # Skip processing the first tab
        
    if skip_last_tab:
        break  # Exit the loop if the last tab is reached
        
    # Determine the column indices dynamically based on the actual number of columns
    num_cols = min(7, len(df.columns))
    
    # Ensure we don't exceed the number of columns in the DataFrame
    columns_to_read = list(range(num_cols))
    
    # Select only the specified columns
    df_selected = df.iloc[:, columns_to_read].copy()
    
    # Add a new column with the tab name to the selected DataFrame using loc
    df_selected.loc[:, 'Year/s'] = sheet_name
    
    # Append the modified DataFrame to the list
    dfs_with_tab_name.append(df_selected)    

In [14]:
# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(dfs_with_tab_name, ignore_index=True)  

In [15]:
# Define a dictionary to map old values to new values
value_mapping = {'Table 1.1': '2003-2004', 'Table 1.2': '2005-2006', 'Table 1.3': '2007-2008', 'Table 1.4': '2009-2010', 'Table 1.5': '2011-2012', 'Table 1.6': '2013-2014', 'Table 1.7': '2015-2016', 'Table 1.8': '2017-2018', 'Table 1.9': '2019-2020', 'Table 1.10': '2020-2021', 'Table 1.11': '2021-2022', 'Explanatory Notes': '', 'tab_name': 'Year/s'}

In [16]:
# Replace values in column 'tab_name' using the value_mapping dictionary
combined_df['Year/s'] = combined_df['Year/s'].replace(value_mapping)

In [17]:
# Remove rows where column 'Year/s' has value '2020-2021' or '2021-2022'
values_to_remove = ['2020-2021', '2021-2022']
cleaned_df = combined_df[~combined_df['Year/s'].isin(values_to_remove)]

In [18]:
# Convert values to numeric data type
cleaned_df.loc[:, 'Wages and salaries'] = cleaned_df['Wages and salaries'].astype(float)
cleaned_df.loc[:, 'Income from  unincorporated business (a)'] = cleaned_df['Income from  unincorporated business (a)'].astype(float)
cleaned_df.loc[:, 'Property income and superannuation'] = cleaned_df['Property income and superannuation'].astype(float)
cleaned_df.loc[:, 'Government pensions and allowances'] = cleaned_df['Government pensions and allowances'].astype(float)
cleaned_df.loc[:, 'Other'] = cleaned_df['Other'].astype(float)
cleaned_df.loc[:, 'All households (b)'] = cleaned_df['All households (b)'].astype(float)
cleaned_df.loc[:, 'Year/s'] = cleaned_df['Year/s'].astype(str)

In [19]:
# Specify the file path for the CSV file
csv_file_path = 'ABS_income_data.csv'  

In [20]:
# Save the DataFrame to a CSV file
cleaned_df.to_csv(csv_file_path, index=False) 

In [21]:
print(f"DataFrame successfully saved to '{csv_file_path}'.")

DataFrame successfully saved to 'ABS_income_data.csv'.


In [22]:
# Create a pivot table with 'Category' as rows, 'Year' as columns, and 'Value' as values
pivot_df = cleaned_df.pivot_table(columns='Year/s', values='All households (b)', aggfunc='sum')


In [23]:
# rename the index as "Household income"
pivot_df = cleaned_df.pivot_table(index='Unnamed: 0', columns='Year/s', values='All households (b)', aggfunc='sum')
pivot_df = pivot_df.rename_axis('Household income')

In [24]:
household_income_df = pivot_df
household_income_df.head()

Year/s,2003-2004,2005-2006,2007-2008,2009-2010,2011-2012,2013-2014,2015-2016,2017-2018,2019-2020
Household income,,,,,,,,,
Estimated number of households in population,7954585.0,8160856.0,8327818.0,8664857.0,8912566.0,9048583.0,9246191.0,9554316.0,10016972.0
Gross disposable income,536356.0,608301.0,725929.0,827246.0,939989.0,1022855.0,1095050.0,1165497.0,1282944.0
Total gross income,711537.0,819132.0,985903.0,1083258.0,1248033.0,1336473.0,1433237.0,1530133.0,1656039.0
Total income payable,175181.0,210831.0,259974.0,256011.0,308044.0,313618.0,338187.0,364636.0,373095.0


# Importing the Population and Suburb data from the ABS
(ED)

In [25]:
# Files to Load
population_data_to_load_2021 = ("../Project_1/2021_GCP_SAL_for_VIC_short-header/2021 Census GCP Suburbs and Localities for VIC/2021Census_G03_VIC_SAL.csv")
suburb_name_data_to_load_2021 = ("../Project_1/2021_GCP_SAL_for_VIC_short-header/Metadata/2021Census_geog_desc_1st_2nd_3rd_release.xlsx")

In [26]:
# Read Population data into Pandas DataFrames
population_data_2021_df = pd.read_csv(population_data_to_load_2021)

# Read population and Suburb data into Pandas DataFrames and specifiy which worksheet number to be read in suburb file
sheet_name = 0
suburb_name_data_2021_df = pd.read_excel("../Project_1/2021_GCP_SAL_for_VIC_short-header/Metadata/2021Census_geog_desc_1st_2nd_3rd_release.xlsx", sheet_name = 5)
# Reference:https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html

FileNotFoundError: [Errno 2] No such file or directory: '../Project_1/2021_GCP_SAL_for_VIC_short-header/2021 Census GCP Suburbs and Localities for VIC/2021Census_G03_VIC_SAL.csv'

In [ ]:
# Test only: Check data load for suburb data
suburb_name_data_2021_df.head()

In [ ]:
# Rename column in Population data for allow DataFrames to merge
population_data_2021_df= population_data_2021_df.rename(columns={"SAL_CODE_2021":"Census_Code_2021"})
population_data_2021_df.head()

In [ ]:
# Merge Population Data and Suburb Data DataFrames
population_and_suburb_merged_df = pd.merge(population_data_2021_df, suburb_name_data_2021_df, how = "left", on=["Census_Code_2021", "Census_Code_2021"])

In [ ]:
# Set Index to AGSS_Code_2021 and rename columns to meaningful headings
population_and_suburb_data_df = population_and_suburb_merged_df.set_index(["AGSS_Code_2021"])
population_and_suburb_data_df = population_and_suburb_data_df.rename(columns={"Total_Total":"Total_Population","Census_Name_2021":"Suburb_Name"})

In [ ]:
# Create final DataFrame with only the required columns
population_and_suburb_df = population_and_suburb_data_df[["Census_Code_2021","Suburb_Name","Total_Population","Area sqkm"]]

In [ ]:
# Test only: Check final DataFrame with required columns
population_and_suburb_df.head()